# Librerías

En este voy a dividir imdb score y rating y voy a intentar que funcione

In [1]:
!pip install selenium

In [2]:
# Descarga automáticamente la versión correcta de chromedriver
!pip install chromedriver_autoinstaller

In [3]:
!pip install lxml

In [4]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

Empezamos por definir la funcion que abra el navegador y haga scroll down

In [5]:
def peliculas(año):
    # Importar las librerías necesarias
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from bs4 import BeautifulSoup as bs
    import time

    # Configuración del navegador
    options = Options()
    options.headless = False  # Se ejecuta con interfaz gráfica
    service = Service()  # Se detecta automáticamente el ChromeDriver
    driver = webdriver.Chrome(service=service, options=options)

    # Abrir la URL con el año especificado
    url = f"https://www.justwatch.com/es/peliculas?release_year_from={año}&release_year_until={año}"
    driver.get(url)
    
    # Esperar a que se muestre el cuadro de cookies
    time.sleep(3)
    
    # Intentar acceder al shadow root y hacer click en "Aceptar todas las cookies"
    try:
        shadow_root = driver.execute_script("return arguments[0].shadowRoot", driver.find_element(By.CSS_SELECTOR, "#usercentrics-root"))
        time.sleep(1)
        shadow_root.find_element(By.CSS_SELECTOR, ".sc-dcJsrY.dQaUXI").click()
    except Exception as e:
        print("No se encontró o no se pudo aceptar las cookies:", e)
    
    # Realizar scroll hasta el final de la página para cargar todo el contenido
    last_height = driver.execute_script("return document.body.scrollHeight")

    
    while True:
        # Hacer scroll hasta el final de la página
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        try:
            # Esperar a que la altura de la página cambie
            WebDriverWait(driver, 3).until(
                lambda driver: driver.execute_script("return document.body.scrollHeight") > last_height
            )
            new_height = driver.execute_script("return document.body.scrollHeight")
            last_height = new_height
            
        except Exception:
            # En caso de no cargarse más contenido, intentar hacer scroll inverso y de nuevo hacia abajo
            try:
                iteration_count = 0
                while iteration_count < 10:
                    for _ in range(5):
                        driver.execute_script("window.scrollBy(0, -400);")
                        time.sleep(0.5)
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    
                    WebDriverWait(driver, 3).until(
                        lambda driver: driver.execute_script("return document.body.scrollHeight") > last_height
                    )
                    new_height = driver.execute_script("return document.body.scrollHeight")
                    
                    if new_height == last_height:
                        break
                    last_height = new_height
                    iteration_count += 1
                    
                if iteration_count == 10:
            except Exception:
            break

    # Obtener el HTML de la página y procesarlo con BeautifulSoup
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    # Extraemos los títulos de las películas buscando las etiquetas 'a' con la clase 'title-list' y nos quedamos con los atributos 'href'
    movie_titles = soup.find_all('a', class_='title-list-grid__item--link', attrs={'href': True})
    movie_title_list = [movie['href'] for movie in movie_titles]

    # Eliminar duplicados
    movie_title_list_unique = list(set(movie_title_list))
    print("Número de películas encontradas:", len(movie_title_list_unique))
        
   
    # Llamar a la función extractor para obtener los datos detallados de cada película
    data = extractor(movie_title_list_unique)
        
    df = pd.DataFrame({
        "title": data["title"],
        "year": data["year"],
        "platforms": data["platforms"],
        "jw_score": data["jw_score"],
        "jw_ratings": data["jw_ratings"],
        "imdb_score": data["imdb_score"],
        "imdb_ratings": data["imdb_ratings"],
        "genres": data["genres"],
        "duration": data["duration"],
        "ageRating": data["ageRating"],
        "countries": data["countries"]
    })
    
    excel_file = f"{año}.xlsx"
    df.to_excel(excel_file, index=False)

    return 'Información Extraída'


In [6]:
def extractor(movie_title_list_unique):
    import requests
    from bs4 import BeautifulSoup as bs
    import time

    # Inicialización de listas para almacenar la información
    title = []
    year = []
    stream = []
    platforms = []
    jw_score = []
    jw_ratings = []
    imdb_score = []
    imdb_ratings = []
    genres_ = []
    genres = []
    duration = []
    ageRating = []
    countries_ = []
    countries = []

    # Usamos un user-agent para que la solicitud parezca provenir de un navegador real y evitar bloqueos
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/120.0.0.0 Safari/537.36"
    }

    # Iteramos sobre la lista de títulos (enlaces)
    for movie in movie_title_list_unique:
        url = f"https://www.justwatch.com{movie}"
        response = requests.get(url, headers=headers)
        soup = bs(response.text, "lxml")

        # Extraemos el título y el año
        title_year = soup.find("h1", class_="title-detail-hero__details__title")
        if title_year:
            title_text = title_year.find(string=True, recursive=False)
            if title_text:
                title.append(title_text.strip())
            else:
                title.append("N/A")
            year_tag = title_year.find("span", class_="release-year")
            if year_tag:
                year.append(year_tag.text.strip())
            else:
                year.append("N/A")
        else:
            title.append("N/A")
            year.append("N/A")

        # Extraemos las plataformas de streaming
        offers_section = soup.find("div", class_="buybox buybox-selector")
        if offers_section:
            offer = offers_section.find("span", class_="offer-container")
            if offer and offer.find("p", class_="offer__label__text"):
                if offer.find("p", class_="offer__label__text").text == "Suscripción":
                    stream.append(offer.find("img")['alt'].strip())
            platforms.append(stream)
            stream = []  # Reiniciamos la lista de plataformas
        else:
            platforms.append(["Nop"])

        # Extraemos las puntuaciones de JustWatch y IMDb
        ratings = soup.find("div", class_="jw-scoring-listing")
        if ratings:
            areJWScore = ratings.find("div", class_="jw-scoring-listing__rating--group jw-scoring-listing__rating--no-link")
            if areJWScore:
                areJWScore2 = areJWScore.find("div")
                if areJWScore2:
                    try:
                        jw_score_, jw_ratings_ = areJWScore2.text.strip().split()
                        jw_score.append(jw_score_)
                        jw_ratings.append(jw_ratings_)
                    except ValueError:
                        jw_score.append("NOP")
                        jw_ratings.append("NOP")
                else:
                    jw_score.append("NOP")
                    jw_ratings.append("NOP")
            else:
                jw_score.append("NOP")
                jw_ratings.append("NOP")
            # IMDb (extraer score y ratings de forma similar)
            areIMDbScore = ratings.find("div", class_="jw-scoring-listing__rating--group jw-scoring-listing__rating--link")
            if areIMDbScore:
                imdb_div = areIMDbScore.find("div")
                if imdb_div:
                    try:
                        imdb_score_, imdb_ratings_ = imdb_div.text.strip().split()
                        imdb_score.append(imdb_score_)
                        imdb_ratings.append(imdb_ratings_)
                    except ValueError:
                        imdb_score.append("NOP")
                        imdb_ratings.append("NOP")
                else:
                    imdb_score.append("NOP")
                    imdb_ratings.append("NOP")
            else:
                imdb_score.append("NOP")
                imdb_ratings.append("NOP")
        else:
            jw_score.append("NOP")
            jw_ratings.append("NOP")
            imdb_score.append("NOP")
            imdb_ratings.append("NOP")
            
        # Extraemos el género de la película
        areGenres = soup.find('h3', class_='poster-detail-infos__subheading', string='Géneros')
        if areGenres:
            genres_ = areGenres.find_next_sibling('div', class_='poster-detail-infos__value').text.strip()
            genres.append(genres_)
        else:
            genres.append("Nop")

        # Extraemos la duración de la película
        isDuration = soup.find('h3', class_='poster-detail-infos__subheading', string='Duración')
        if isDuration:
            duration.append(isDuration.find_next_sibling('div', class_='poster-detail-infos__value').text.strip())
        else:
            duration.append("Nop")

        # Extraemos la clasificación por edades
        isAgeRating = soup.find('h3', class_='poster-detail-infos__subheading', string='Clasificación por edades')
        if isAgeRating:
            ageRating.append(isAgeRating.find_next_sibling('div', class_='poster-detail-infos__value').text.strip())
        else:
            ageRating.append(0)

        # Extraemos el país de producción
        areCountries = soup.find('h3', class_='poster-detail-infos__subheading', string='País de producción')
        if areCountries:
            countries_ = areCountries.find_next_sibling('div', class_='poster-detail-infos__value').text.strip()
            countries.append(countries_)
        else:
            countries.append("Nop")

        # Esperamos 0.5 segundos entre peticiones para no sobrecargar el servidor
        time.sleep(0.5)

    # Se retorna un diccionario con toda la información extraída
    return {
        "title": title,
        "year": year,
        "platforms": platforms,
        "jw_score": jw_score,
        "jw_ratings": jw_ratings,
        "imdb_score": imdb_score,
        "imdb_ratings": imdb_ratings,
        "genres": genres,
        "duration": duration,
        "ageRating": ageRating,
        "countries": countries
    }

In [ ]:
peliculas(2023)

Se alcanzó el límite de 10 iteraciones.
Número de películas encontradas: 1900
